In [2]:
import HandDetectionClass as hd
import cv2
import numpy as np
import pandas as pd
import time
import autopy
import mediapipe as mp
import math

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [7]:
#We will control the volume by the thumb and index finger
previous_time = 0
current_time = 0
cap = cv2.VideoCapture(0)
width, height = cap.get(3), cap.get(4) #640, 480
detector = hd.HandDetectionClass(max_hands=1, min_det_con=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))


#volume.GetMute()
#volume.GetMasterVolumeLevel()
#print(volume.GetVolumeRange()) #Range: (-65.25, 0.0, 0.03125), 0 is maximum, -65.25 is minimum
a = -65.25
b = 0
while True:
    retval, image = cap.read()
    image = detector.findHands(image)
    landmarks_list = detector.findPosition(image, 0, False)
    if len(landmarks_list) != 0:
        #Thumb ID is 4, Index ID is 8, Middle Finger ID is 12 etc.. others from Mediapipe google
        #print(landmarks_list[4], landmarks_list[8])
        
        thumb_x, thumb_y = landmarks_list[4][1], landmarks_list[4][2]
        index_x, index_y = landmarks_list[8][1], landmarks_list[8][2]
        point1, point2 = (thumb_x, thumb_y), (index_x, index_y)
        cv2.circle(image, (thumb_x, thumb_y), 10, (0,0,0), 4)
        cv2.circle(image, (index_x, index_y), 10, (0,0,0), 4)
        cv2.line(image, point1, point2, (255,0,0),2)
        #Getting the line's center:
        center_x, center_y = int((thumb_x+index_x)/2), int((thumb_y+index_y)/2)
        cv2.circle(image, (center_x, center_y), 5, (0,0,0), -1)
        distance = math.hypot(index_x-thumb_x, index_y-thumb_y)
        print(distance)
        ### That's the first method, pure mathematics :)
        x = distance
        xmax = 190
        xmin = 20
#         if distance < 20:
#             cv2.circle(image, (center_x, center_y), 5, (255,255,255), -1)
#             volume.SetMasterVolumeLevel(-65.25, None)
#         elif distance > 190: #it's changable, depends on how far is your hand from the camera etc ..
#             volume.SetMasterVolumeLevel(0.0, None)
#         else:
#             #formula from: https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range
#             xnorm = (b-a)*(x - xmin)/(xmax-xmin)+a
#             volume.SetMasterVolumeLevel(xnorm, None)
         ### 2nd method:  
        vol = np.interp(distance, [xmin, xmax],[a, b])
        volume.SetMasterVolumeLevel(vol, None)
        #We can graph a line which indicates the volume bar
        vol_percentage = np.interp(distance, [xmin, xmax],[0, 100])
        vol_bar = np.interp(distance, [xmin, xmax],[500, 640])
        cv2.rectangle(image, (500, 435), (int(vol_bar), 445), (180,100,0), -1)
        #cv2.line(image, (550, 450), (int(vol_bar), 450), (180,100,0), 3)
        cv2.putText(image, str(int(vol_percentage)), (550,475), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)        

    
    current_time = time.time()
    fps = 1/(current_time-previous_time)
    previous_time = current_time
    cv2.putText(image, str(int(fps)), (5,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)        
    cv2.imshow('Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

224.83104767802868
200.14494747557333
218.28879952943075
227.705950734714
228.32433072276814
221.7408397206072
213.60009363293827
197.436065600994
192.4421991144354
163.48700254148645
102.45974819410792
99.72462083156796
83.5224520712844
35.4682957019364
34.20526275297414
33.24154027718932
29.732137494637012
28.178005607210743
32.89376840679706
73.00684899377592
122.09013064126027
120.90492132250037
168.0773631397161
170.07351351694948
179.8471573308847
177.6119365358083
175.45369759569047
179.9222054111165
178.51890656174209
178.70926109186394
179.42686532400882
183.35757415498276
185.64751546950473
184.70787747142782
183.3712082089225
186.62529303392938
181.4662503056698
183.57559750685817
152.61061562027723
7.810249675906656
4.123105625617661
